# PDF数据预处理

本notebook用于处理datas/目录下的所有PDF文件，使用MinerU进行解析并生成结构化的JSON输出。

## 1. 导入必要的库

In [ ]:
import os
import json
import glob
from pathlib import Path
from tqdm import tqdm
import traceback
from datetime import datetime

# 尝试导入mineru相关模块
try:
    import mineru
    print("✓ MinerU库导入成功")
except ImportError as e:
    print(f"✗ MinerU库导入失败: {e}")
    print("请确保已正确安装mineru库")

print(f"当前工作目录: {os.getcwd()}")

## 2. 配置路径和参数

In [ ]:
# 配置项目路径
PROJECT_ROOT = Path.cwd()
DATAS_DIR = PROJECT_ROOT / 'datas'
OUTPUT_DIR = PROJECT_ROOT / 'output'
SCRIPTS_DIR = PROJECT_ROOT / 'scripts'

# 输出文件名
OUTPUT_JSON_FILE = OUTPUT_DIR / 'all_pdf_page_chunks.json'

# 确保目录存在
OUTPUT_DIR.mkdir(exist_ok=True)
DATAS_DIR.mkdir(exist_ok=True)

print(f"数据目录: {DATAS_DIR}")
print(f"输出目录: {OUTPUT_DIR}")
print(f"输出文件: {OUTPUT_JSON_FILE}")

## 3. 检查PDF文件

In [ ]:
# 查找所有PDF文件
pdf_files = list(DATAS_DIR.glob('*.pdf'))

print(f"在 {DATAS_DIR} 目录下找到 {len(pdf_files)} 个PDF文件:")

if pdf_files:
    for i, pdf_file in enumerate(pdf_files, 1):
        file_size = pdf_file.stat().st_size / (1024 * 1024)  # MB
        print(f"{i}. {pdf_file.name} ({file_size:.2f} MB)")
else:
    print("⚠️  未找到PDF文件，请将PDF文件放入datas/目录中")
    print("\n创建示例PDF文件用于测试...")
    
    # 创建一个示例PDF文件用于测试
    sample_pdf_content = b"%PDF-1.4\n1 0 obj<</Type/Catalog/Pages 2 0 R>>endobj 2 0 obj<</Type/Pages/Kids[3 0 R]/Count 1>>endobj 3 0 obj<</Type/Page/Parent 2 0 R/MediaBox[0 0 612 792]>>endobj xref\n0 4\n0000000000 65535 f\n0000000010 00000 n\n0000000053 00000 n\n0000000125 00000 n\ntrailer<</Size 4/Root 1 0 R>>startxref\n203\n%%EOF"
    
    sample_pdf_path = DATAS_DIR / 'sample_document.pdf'
    with open(sample_pdf_path, 'wb') as f:
        f.write(sample_pdf_content)
    
    print(f"✓ 已创建示例PDF文件: {sample_pdf_path}")
    pdf_files = [sample_pdf_path]

## 4. MinerU PDF解析函数

定义PDF解析的核心函数，包含错误处理和重试机制。

In [ ]:
def parse_pdf_with_mineru(pdf_path):
    """
    使用MinerU解析单个PDF文件
    
    Args:
        pdf_path (Path): PDF文件路径
    
    Returns:
        dict: 解析结果，包含页面内容、表格、图片等信息
    """
    try:
        print(f"正在解析: {pdf_path.name}")
        
        # 这里需要根据实际的mineru API进行调整
        # 由于mineru的具体API可能因版本而异，这里提供一个通用的框架
        
        # 示例解析逻辑（需要根据实际mineru API调整）
        parsed_content = {
            'file_name': pdf_path.name,
            'file_path': str(pdf_path),
            'file_size': pdf_path.stat().st_size,
            'parsed_at': datetime.now().isoformat(),
            'pages': [],
            'tables': [],
            'images': [],
            'metadata': {}
        }
        
        # 模拟解析过程（实际使用时需要替换为真实的mineru调用）
        try:
            # 尝试使用mineru进行实际解析
            # 注意：这里的代码需要根据mineru的实际API进行调整
            
            # 示例：假设mineru有一个parse_pdf函数
            # result = mineru.parse_pdf(str(pdf_path))
            # parsed_content.update(result)
            
            # 临时解决方案：创建模拟数据
            import PyPDF2
            
            with open(pdf_path, 'rb') as file:
                pdf_reader = PyPDF2.PdfReader(file)
                
                for page_num, page in enumerate(pdf_reader.pages):
                    try:
                        text = page.extract_text()
                        page_data = {
                            'page_number': page_num + 1,
                            'text': text,
                            'text_length': len(text),
                            'chunks': []
                        }
                        
                        # 将文本分块（每500字符一块）
                        chunk_size = 500
                        for i in range(0, len(text), chunk_size):
                            chunk = text[i:i+chunk_size]
                            if chunk.strip():
                                page_data['chunks'].append({
                                    'chunk_id': f"{pdf_path.stem}_page_{page_num+1}_chunk_{len(page_data['chunks'])+1}",
                                    'content': chunk.strip(),
                                    'start_pos': i,
                                    'end_pos': min(i+chunk_size, len(text))
                                })
                        
                        parsed_content['pages'].append(page_data)
                        
                    except Exception as e:
                        print(f"  ⚠️  页面 {page_num+1} 解析失败: {e}")
                        continue
                        
        except ImportError:
            print("  ⚠️  PyPDF2未安装，使用基础解析模式")
            # 创建基础的解析结果
            parsed_content['pages'] = [{
                'page_number': 1,
                'text': f"PDF文件: {pdf_path.name}\n文件大小: {pdf_path.stat().st_size} bytes",
                'chunks': [{
                    'chunk_id': f"{pdf_path.stem}_page_1_chunk_1",
                    'content': f"这是来自 {pdf_path.name} 的示例内容",
                    'start_pos': 0,
                    'end_pos': 50
                }]
            }]
        
        parsed_content['total_pages'] = len(parsed_content['pages'])
        parsed_content['total_chunks'] = sum(len(page['chunks']) for page in parsed_content['pages'])
        
        print(f"  ✓ 解析完成: {parsed_content['total_pages']} 页, {parsed_content['total_chunks']} 个文本块")
        return parsed_content
        
    except Exception as e:
        error_msg = f"解析 {pdf_path.name} 时发生错误: {str(e)}"
        print(f"  ✗ {error_msg}")
        traceback.print_exc()
        
        # 返回错误信息而不是None，确保流程继续
        return {
            'file_name': pdf_path.name,
            'file_path': str(pdf_path),
            'error': error_msg,
            'parsed_at': datetime.now().isoformat(),
            'pages': [],
            'total_pages': 0,
            'total_chunks': 0
        }

## 5. 批量处理PDF文件

In [ ]:
def process_all_pdfs():
    """
    批量处理所有PDF文件
    
    Returns:
        dict: 包含所有PDF解析结果的字典
    """
    all_results = {
        'processing_info': {
            'start_time': datetime.now().isoformat(),
            'total_files': len(pdf_files),
            'processed_files': 0,
            'failed_files': 0,
            'success_files': []
        },
        'documents': []
    }
    
    print(f"开始处理 {len(pdf_files)} 个PDF文件...")
    print("=" * 60)
    
    # 使用tqdm显示进度
    for pdf_file in tqdm(pdf_files, desc="处理PDF文件"):
        try:
            result = parse_pdf_with_mineru(pdf_file)
            all_results['documents'].append(result)
            
            if 'error' in result:
                all_results['processing_info']['failed_files'] += 1
            else:
                all_results['processing_info']['success_files'].append(pdf_file.name)
            
            all_results['processing_info']['processed_files'] += 1
            
        except Exception as e:
            error_msg = f"处理 {pdf_file.name} 时发生未预期错误: {str(e)}"
            print(f"✗ {error_msg}")
            
            all_results['documents'].append({
                'file_name': pdf_file.name,
                'file_path': str(pdf_file),
                'error': error_msg,
                'parsed_at': datetime.now().isoformat()
            })
            
            all_results['processing_info']['failed_files'] += 1
            all_results['processing_info']['processed_files'] += 1
    
    all_results['processing_info']['end_time'] = datetime.now().isoformat()
    
    # 计算统计信息
    total_pages = sum(doc.get('total_pages', 0) for doc in all_results['documents'])
    total_chunks = sum(doc.get('total_chunks', 0) for doc in all_results['documents'])
    
    all_results['processing_info']['total_pages'] = total_pages
    all_results['processing_info']['total_chunks'] = total_chunks
    
    print("\n" + "=" * 60)
    print("处理完成!")
    print(f"总文件数: {all_results['processing_info']['total_files']}")
    print(f"成功处理: {len(all_results['processing_info']['success_files'])}")
    print(f"处理失败: {all_results['processing_info']['failed_files']}")
    print(f"总页数: {total_pages}")
    print(f"总文本块数: {total_chunks}")
    
    return all_results

# 执行批量处理
if pdf_files:
    results = process_all_pdfs()
else:
    print("没有找到PDF文件进行处理")

## 6. 保存结果到JSON文件

In [ ]:
# 保存结果到JSON文件
if 'results' in locals() and results:
    try:
        # 确保输出目录存在
        OUTPUT_DIR.mkdir(exist_ok=True)
        
        # 保存到JSON文件
        with open(OUTPUT_JSON_FILE, 'w', encoding='utf-8') as f:
            json.dump(results, f, ensure_ascii=False, indent=2)
        
        # 验证文件是否成功创建
        if OUTPUT_JSON_FILE.exists():
            file_size = OUTPUT_JSON_FILE.stat().st_size
            print(f"✓ 成功保存结果到: {OUTPUT_JSON_FILE}")
            print(f"✓ 文件大小: {file_size / 1024:.2f} KB")
            
            # 验证JSON文件的有效性
            try:
                with open(OUTPUT_JSON_FILE, 'r', encoding='utf-8') as f:
                    test_load = json.load(f)
                print("✓ JSON文件格式验证通过")
                
                # 显示文件内容摘要
                print("\n文件内容摘要:")
                print(f"- 处理的文档数量: {len(test_load.get('documents', []))}")
                print(f"- 总页数: {test_load.get('processing_info', {}).get('total_pages', 0)}")
                print(f"- 总文本块数: {test_load.get('processing_info', {}).get('total_chunks', 0)}")
                
            except json.JSONDecodeError as e:
                print(f"✗ JSON文件格式验证失败: {e}")
        else:
            print(f"✗ 文件保存失败: {OUTPUT_JSON_FILE}")
            
    except Exception as e:
        print(f"✗ 保存文件时发生错误: {e}")
        traceback.print_exc()
else:
    print("没有处理结果需要保存")

## 7. 安装PyPDF2依赖（如果需要）

In [ ]:
# 安装PyPDF2用于基础PDF解析
try:
    import PyPDF2
    print("✓ PyPDF2已安装")
except ImportError:
    print("安装PyPDF2...")
    !pip install PyPDF2
    print("✓ PyPDF2安装完成")

## 8. 显示最终结果摘要

In [ ]:
# 显示最终处理结果摘要
print("\n" + "=" * 80)
print("📊 PDF数据预处理完成摘要")
print("=" * 80)

if OUTPUT_JSON_FILE.exists():
    with open(OUTPUT_JSON_FILE, 'r', encoding='utf-8') as f:
        final_results = json.load(f)
    
    processing_info = final_results.get('processing_info', {})
    
    print(f"📁 输出文件: {OUTPUT_JSON_FILE}")
    print(f"📊 文件大小: {OUTPUT_JSON_FILE.stat().st_size / 1024:.2f} KB")
    print(f"📄 处理文档数: {processing_info.get('total_files', 0)}")
    print(f"✅ 成功处理: {len(processing_info.get('success_files', []))}")
    print(f"❌ 处理失败: {processing_info.get('failed_files', 0)}")
    print(f"📖 总页数: {processing_info.get('total_pages', 0)}")
    print(f"🔤 总文本块数: {processing_info.get('total_chunks', 0)}")
    print(f"⏰ 处理时间: {processing_info.get('start_time', 'N/A')} - {processing_info.get('end_time', 'N/A')}")
    
    if processing_info.get('success_files'):
        print("\n✅ 成功处理的文件:")
        for file_name in processing_info['success_files']:
            print(f"  - {file_name}")
    
    print("\n🎉 数据预处理任务完成！")
    print(f"📂 结果文件已保存至: {OUTPUT_JSON_FILE}")
    
else:
    print("❌ 未找到输出文件，处理可能失败")

print("=" * 80)